In [3]:
import pandas as pd

# Load dataset
df = pd.read_csv("Untitled spreadsheet - Data Tape.csv")

# Print column names to identify mismatches
print(df.columns)


Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Loan Categories', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 182', 'Unnamed: 183', 'Unnamed: 184', 'Unnamed: 185',
       'Unnamed: 186', 'Unnamed: 187', 'Unnamed: 188', 'Unnamed: 189',
       'Unnamed: 190', 'Capital Calcs'],
      dtype='object', length=192)


In [4]:
import pandas as pd

# Load dataset, using row 6 (index 5) as column headers
df = pd.read_csv("Untitled spreadsheet - Data Tape.csv", header=5)

# Strip any unwanted spaces from column names
df.columns = df.columns.str.strip()

# Print column names to confirm
print(df.columns)

# Now, select the needed columns
df = df[['Borrower FICO', 'Qualifying DTI', 'Borrower Verified Income', 'Defaulted']].dropna()

# Preview data
df.head()


Index(['Unnamed: 0', 'Include?', 'LoanID', 'LD LoanID', 'Loan Category',
       'Product Type', 'Original Loan Amount', 'Loan Product', 'Loan Type',
       'Deferred Loan',
       ...
       'Minimum of (i) and (ii)', 'Excess', 'Unnamed: 184', 'FICO <= 669',
       'Unnamed: 186', 'FICO <= 699', 'Unnamed: 188', 'FICO <= 759',
       'Unnamed: 190', 'Sum of Sqaures'],
      dtype='object', length=192)


,Borrower FICO,Qualifying DTI,Borrower Verified Income,Defaulted
1,689,36.3%,15521.42,0.0
32,683,35.4%,5731.21,0.0
54,655,37.1%,5248.83,0.0
67,693,38.4%,11192.42,0.0
85,651,49.3%,6887.07,0.0


In [5]:
# Convert 'Qualifying DTI' from percentage to float
df['Qualifying DTI'] = df['Qualifying DTI'].str.replace('%', '').astype(float) / 100

# Convert 'Borrower Verified Income' to binary (1 = Verified, 0 = Unverified)
df['Borrower Verified Income'] = (df['Borrower Verified Income'] > 0).astype(int)

# Print to confirm
print(df.head())


    Borrower FICO  Qualifying DTI  Borrower Verified Income  Defaulted
1             689           0.363                         1        0.0
32            683           0.354                         1        0.0
54            655           0.371                         1        0.0
67            693           0.384                         1        0.0
85            651           0.493                         1        0.0


In [7]:
import pandas as pd
import numpy as np

# Load cleaned dataset
df = pd.read_csv("Clean Untitled spreadsheet - Data Tape.csv", header=5)

# Strip spaces from column names
df.columns = df.columns.str.strip()

# Convert 'Qualifying DTI' from percentage to float
df['Qualifying DTI'] = df['Qualifying DTI'].astype(str).str.replace('%', '').astype(float) / 100

# Convert 'Borrower Verified Income' to binary (1 = Verified, 0 = Unverified)
df['Borrower Verified Income'] = (df['Borrower Verified Income'] > 0).astype(int)

# Drop missing values in relevant columns
df = df[['Borrower FICO', 'Qualifying DTI', 'Borrower Verified Income', 'Defaulted']].dropna()

# Define fuzzy sets
def fuzzy_fico(fico):
    if fico > 720:
        return 2  # Low Risk
    elif fico >= 650:
        return 1  # Medium Risk
    else:
        return 0  # High Risk

def fuzzy_dti(dti):
    if dti > 0.45:   # DTI > 45% is Critical
        return 0
    elif dti > 0.30: # DTI 30-45% is Manageable
        return 1
    else:            # DTI < 30% is Affordable
        return 2

def fuzzy_income(verified):
    return 2 if verified == 1 else 1  # 2 = Verified, 1 = Unverified

# Apply fuzzy classification
df['fico_level'] = df['Borrower FICO'].apply(fuzzy_fico)
df['dti_level'] = df['Qualifying DTI'].apply(fuzzy_dti)
df['income_level'] = df['Borrower Verified Income'].apply(fuzzy_income)

# Assign each loan to a state in 3x3 Gridworld (State = FICO x DTI)
df['state'] = df.apply(lambda row: row['fico_level'] * 3 + row['dti_level'], axis=1)

# Optional: Assign X, Y grid coordinates for visualization
df['grid_x'] = df['fico_level']
df['grid_y'] = df['dti_level']

# Preview cleaned dataset with assigned states
print(df[['Borrower FICO', 'Qualifying DTI', 'Borrower Verified Income', 'fico_level', 'dti_level', 'state']].head())


   Borrower FICO  Qualifying DTI  Borrower Verified Income  fico_level  \
0            720           0.000                         0           1   
1            689           0.363                         1           1   
2            702           0.000                         0           1   
3            720           0.000                         0           1   
4            817           0.000                         0           2   

   dti_level  state  
0          2    5.0  
1          1    4.0  
2          2    5.0  
3          2    5.0  
4          2    8.0  


In [8]:
# Check unique values in DTI
print(df['Qualifying DTI'].unique())


[0.    0.363 0.349 0.311 0.463 0.322 0.231 0.27  0.28  0.221 0.354 0.156
 0.32  0.426 0.361 0.371 0.274 0.486 0.384 0.323 0.497 0.402 0.493 0.358
 0.453 0.102 0.192 0.336 0.203 0.309 0.388 0.488 0.498 0.548 0.473 0.258
 0.352 0.219 0.414 0.123 0.469 0.353 0.224 0.445 0.318 0.098 0.333 0.161
 0.137 0.504 0.484 0.39  0.144 0.29 ]


In [16]:
import pandas as pd
import numpy as np

# Load cleaned dataset (with correct DTI values)
df = pd.read_csv("Clean Untitled spreadsheet - Data Tape.csv", header=5)

# Strip spaces from column names
df.columns = df.columns.str.strip()

# Convert 'Qualifying DTI' from percentage string to float (ensuring correct values)
df['Qualifying DTI'] = df['Qualifying DTI'].astype(str).str.replace('%', '').astype(float)

# Convert 'Borrower Verified Income' to binary (1 = Verified, 0 = Unverified)
df['Borrower Verified Income'] = (df['Borrower Verified Income'] > 0).astype(int)

# Drop missing values in relevant columns
df = df[['Borrower FICO', 'Qualifying DTI', 'Borrower Verified Income', 'Defaulted']].dropna()

# Define fuzzy sets
def fuzzy_fico(fico):
    if fico > 720:
        return 2  # Low Risk
    elif fico >= 650:
        return 1  # Medium Risk
    else:
        return 0  # High Risk

def fuzzy_dti(dti):
    if dti > 45:   # DTI > 45% is Critical
        return 0
    elif dti > 30: # DTI 30-45% is Manageable
        return 1
    else:          # DTI < 30% is Affordable
        return 2

def fuzzy_income(verified):
    return 2 if verified == 1 else 1  # 2 = Verified, 1 = Unverified

# Apply fuzzy classification
df['fico_level'] = df['Borrower FICO'].apply(fuzzy_fico)
df['dti_level'] = df['Qualifying DTI'].apply(fuzzy_dti)
df['income_level'] = df['Borrower Verified Income'].apply(fuzzy_income)

# Assign each loan to a state in 3x3 Gridworld (State = FICO x DTI)
df['state'] = df.apply(lambda row: row['fico_level'] * 3 + row['dti_level'], axis=1).astype(int)

# Optional: Assign X, Y grid coordinates for visualization
df['grid_x'] = df['fico_level'].astype(int)
df['grid_y'] = df['dti_level'].astype(int)

# Preview cleaned dataset with assigned states (non-decimal values)
print(df[['Borrower FICO', 'Qualifying DTI', 'Borrower Verified Income', 'fico_level', 'dti_level', 'state']].head(100))


    Borrower FICO  Qualifying DTI  Borrower Verified Income  fico_level  \
0             720             0.0                         0           1   
1             689            36.3                         1           1   
2             702             0.0                         0           1   
3             720             0.0                         0           1   
4             817             0.0                         0           2   
..            ...             ...                       ...         ...   
95            826             0.0                         0           2   
96            833             0.0                         0           2   
97            800             0.0                         0           2   
98            698            10.2                         0           1   
99            822             0.0                         0           2   

    dti_level  state  
0           2      5  
1           1      4  
2           2      5  
3      

In [17]:
# Show only rows where DTI is greater than 0
print(df[df['Qualifying DTI'] > 0].head(10))


    Borrower FICO  Qualifying DTI  Borrower Verified Income  Defaulted  \
1             689            36.3                         1        0.0   
9             660            34.9                         0        0.0   
10            661            31.1                         0        0.0   
14            651            46.3                         0        0.0   
19            579            32.2                         0        0.0   
20            695            23.1                         0        0.0   
21            658            27.0                         0        0.0   
25            694            28.0                         0        0.0   
28            655            22.1                         0        0.0   
32            683            35.4                         1        0.0   

    fico_level  dti_level  income_level  state  grid_x  grid_y  
1            1          1             2      4       1       1  
9            1          1             1      4       1 

In [18]:
# Define action mapping
ACTIONS = {0: "Approve", 1: "Investigate", 2: "Deny"}

# Assign rewards based on action & default status
def get_reward(action, defaulted):
    if action == 0:  # Approve
        return 10 if defaulted == 0 else -20  # High reward or heavy penalty
    elif action == 1:  # Investigate
        return 5 if defaulted == 0 else -5  # Moderate reward/penalty
    else:  # Deny
        return 0  # Neutral

# Apply action-reward logic to the dataset
df['reward_approve'] = df['Defaulted'].apply(lambda x: get_reward(0, x))
df['reward_investigate'] = df['Defaulted'].apply(lambda x: get_reward(1, x))
df['reward_deny'] = 0  # Always neutral

# Preview dataset with rewards
print(df[['state', 'Defaulted', 'reward_approve', 'reward_investigate', 'reward_deny']].head())


   state  Defaulted  reward_approve  reward_investigate  reward_deny
0      5        0.0              10                   5            0
1      4        0.0              10                   5            0
2      5        0.0              10                   5            0
3      5        0.0              10                   5            0
4      8        0.0              10                   5            0


In [19]:
from collections import defaultdict

# Initialize transition matrix
transitions = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

# Count transitions from (state, action) → next state
for _, row in df.iterrows():
    current_state = row['state']
    
    # Check next state logic (simulate movement based on default history)
    if row['Defaulted'] == 1:
        next_state = 0  # Heavy penalty → Reset to State 0
    else:
        next_state = current_state  # Stay in the same state
    
    # Update transition counts
    transitions[current_state][0][next_state] += 1  # Approve
    transitions[current_state][1][next_state] += 1  # Investigate
    transitions[current_state][2][current_state] += 1  # Deny (stays in the same state)

# Normalize counts to get probabilities
transition_probs = {}
for state in transitions:
    transition_probs[state] = {}
    for action in transitions[state]:
        total_transitions = sum(transitions[state][action].values())
        transition_probs[state][action] = {s_prime: count / total_transitions for s_prime, count in transitions[state][action].items()}

# Print transition probabilities
import pprint
pprint.pprint(transition_probs)


{np.float64(1.0): {0: {np.float64(1.0): 1.0},
                   1: {np.float64(1.0): 1.0},
                   2: {np.float64(1.0): 1.0}},
 np.float64(2.0): {0: {np.float64(2.0): 1.0},
                   1: {np.float64(2.0): 1.0},
                   2: {np.float64(2.0): 1.0}},
 np.float64(3.0): {0: {np.float64(3.0): 1.0},
                   1: {np.float64(3.0): 1.0},
                   2: {np.float64(3.0): 1.0}},
 np.float64(4.0): {0: {np.float64(4.0): 1.0},
                   1: {np.float64(4.0): 1.0},
                   2: {np.float64(4.0): 1.0}},
 np.float64(5.0): {0: {np.float64(5.0): 1.0},
                   1: {np.float64(5.0): 1.0},
                   2: {np.float64(5.0): 1.0}},
 np.float64(6.0): {0: {np.float64(6.0): 1.0},
                   1: {np.float64(6.0): 1.0},
                   2: {np.float64(6.0): 1.0}},
 np.float64(8.0): {0: {np.float64(8.0): 1.0},
                   1: {np.float64(8.0): 1.0},
                   2: {np.float64(8.0): 1.0}}}


In [21]:
import random

# Initialize transition matrix
transitions = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

# Count transitions from (state, action) → next state
for _, row in df.iterrows():
    current_state = int(row['state'])  # Ensure it's an integer
    
    # Introduce dynamic next-state movement
    if row['Defaulted'] == 1:
        next_state = 0  # Default → Move to worst state (state 0)
    else:
        # Borrowers may move slightly (randomness)
        move = random.choice([-1, 0, 1])  # Move left (-1), stay (0), or move right (+1)
        next_state = max(0, min(8, current_state + move))  # Ensure within state range (0-8)

    # Update transition counts
    transitions[current_state][0][next_state] += 1  # Approve
    transitions[current_state][1][next_state] += 1  # Investigate
    transitions[current_state][2][current_state] += 1  # Deny (always stays same)

# Normalize counts to get probabilities
transition_probs = {}
for state in transitions:
    transition_probs[state] = {}
    for action in transitions[state]:
        total_transitions = sum(transitions[state][action].values())
        transition_probs[state][action] = {
            s_prime: round(count / total_transitions, 3) for s_prime, count in transitions[state][action].items()
        }

# Print improved transition probabilities
import pprint
pprint.pprint(transition_probs)


{1: {0: {0: 0.25, 1: 0.25, 2: 0.5}, 1: {0: 0.25, 1: 0.25, 2: 0.5}, 2: {1: 1.0}},
 2: {0: {1: 0.2, 2: 0.6, 3: 0.2}, 1: {1: 0.2, 2: 0.6, 3: 0.2}, 2: {2: 1.0}},
 3: {0: {2: 0.455, 3: 0.364, 4: 0.182},
     1: {2: 0.455, 3: 0.364, 4: 0.182},
     2: {3: 1.0}},
 4: {0: {3: 0.25, 4: 0.3, 5: 0.45}, 1: {3: 0.25, 4: 0.3, 5: 0.45}, 2: {4: 1.0}},
 5: {0: {4: 0.344, 5: 0.23, 6: 0.426},
     1: {4: 0.344, 5: 0.23, 6: 0.426},
     2: {5: 1.0}},
 6: {0: {5: 1.0}, 1: {5: 1.0}, 2: {6: 1.0}},
 8: {0: {7: 0.304, 8: 0.696}, 1: {7: 0.304, 8: 0.696}, 2: {8: 1.0}}}


In [25]:
print(df.columns)


Index(['Borrower FICO', 'Qualifying DTI', 'Borrower Verified Income',
       'Defaulted', 'fico_level', 'dti_level', 'income_level', 'state',
       'grid_x', 'grid_y', 'reward_approve', 'reward_investigate',
       'reward_deny'],
      dtype='object')


In [26]:
# Ensure column names are consistent (lowercase)
df.columns = df.columns.str.strip().str.lower()  # Fix extra spaces & convert to lowercase

# Fix reward column references by making them lowercase
reward_column = f"reward_{ACTIONS[a].lower()}"

# Check if the column exists before accessing
if reward_column in df.columns:
    reward_value = df.loc[df['state'] == s, reward_column].mean()
else:
    reward_value = 0  # Default to 0 if column is missing


In [27]:
expected_columns = ["reward_approve", "reward_investigate", "reward_deny"]
missing_columns = [col for col in expected_columns if col not in df.columns]

if missing_columns:
    print(f"⚠️ Missing columns: {missing_columns}")
else:
    print("✅ All reward columns exist!")


✅ All reward columns exist!


In [28]:
import numpy as np

# Define discount factor and convergence threshold
gamma = 0.9  
theta = 0.0001  

# Initialize Value Function (V) for all states (0-8) as zero
V = {s: 0 for s in range(9)}

# Run Value Iteration until convergence
while True:
    delta = 0  # Change tracker
    new_V = V.copy()

    for s in range(9):  # Loop over all states
        max_value = float('-inf')

        for a in range(3):  # Loop over actions (0=Approve, 1=Investigate, 2=Deny)
            expected_value = 0  # Start with zero

            if s in transition_probs and a in transition_probs[s]:  # Ensure valid state-action pair
                for s_prime in transition_probs[s][a]:
                    # Ensure consistent reward column names
                    reward_column = f"reward_{ACTIONS[a].lower()}"
                    
                    # Check if column exists, else default to 0
                    reward_value = df.loc[df['state'] == s, reward_column].mean() if reward_column in df.columns else 0

                    if np.isnan(reward_value):  # Handle missing rewards
                        reward_value = 0  

                    # Compute expected value
                    expected_value += transition_probs[s][a][s_prime] * (reward_value + gamma * V.get(s_prime, 0))

            if expected_value > max_value:
                max_value = expected_value

        # Update Value Function
        new_V[s] = max_value
        delta = max(delta, abs(V[s] - new_V[s]))

    V = new_V  # Apply update

    if delta < theta:  # Check for convergence
        break

# Compute Optimal Policy (Best Action for Each State)
policy = {}
for s in range(9):
    if s in transition_probs:
        best_action = max(range(3), key=lambda a: sum(
            transition_probs[s][a].get(s_prime, 0) * 
            (df.loc[df['state'] == s, f"reward_{ACTIONS[a].lower()}"].mean() + gamma * V.get(s_prime, 0))
            for s_prime in transition_probs[s][a]
        ))
        policy[s] = ACTIONS[best_action]
    else:
        policy[s] = "No Action"  # If state is missing, assign a default action

# Print Optimal Value Function & Policy
print("\n✅ Optimal Value Function (V*):")
for s in range(9):
    print(f"State {s}: {V[s]:.3f}")

print("\n✅ Optimal Policy (Best Action in Each State):")
for s in range(9):
    print(f"State {s}: {policy[s]}")



✅ Optimal Value Function (V*):
State 0: 0.000
State 1: 57.765
State 2: 77.262
State 3: 84.127
State 4: 91.077
State 5: 92.734
State 6: 93.461
State 7: 0.000
State 8: 26.767

✅ Optimal Policy (Best Action in Each State):
State 0: No Action
State 1: Approve
State 2: Approve
State 3: Approve
State 4: Approve
State 5: Approve
State 6: Approve
State 7: No Action
State 8: Approve
